## 0.0 Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
predictions = pd.read_pickle('../processed_data/history_data.pkl')


In [7]:
predictions['basecase_revenue'] = \
 (np.power(10, predictions['forecast_cons_12m']) + 1) * predictions['forecast_price_energy_p1'] + \
 predictions['forecast_meter_rent_12m']

# Taking churn into account
predictions['basecase_revenue_after_churn'] = predictions['basecase_revenue'] * (1 - 0.919 * predictions['churn'])

KeyError: 'forecast_cons_12m'

In [4]:
def get_rev_delta(pred: pd.DataFrame, cutoff: float=0.5, discount: float=0.2) -> float:
 """
 Get the delta of revenues for offering discount for all customers with predicted churn risk >= cutoff
 """
 pred['discount_revenue'] = pred['basecase_revenue_after_churn']
 # Churn predicted => discount is given => customer stays for full year, independent of whether the prediction
 # (false positive, "free"/unnecessary discount given) or correct
 pred.loc[pred['predict_prob'] >= cutoff, 'discount_revenue'] = pred['basecase_revenue'] * (1 - discount)
 # Save the revenue delta for each customer in a separate column
 pred['revenue_delta'] = pred['discount_revenue'] - pred['basecase_revenue_after_churn']
 return pred['revenue_delta'].sum()

In [5]:
rev_deltas = pd.Series({cutoff: get_rev_delta(predictions, cutoff=cutoff) for cutoff in np.arange(0, 1, 0.01)})
def plot_tradeoff(rev_deltas: pd.Series):
 # Plot the revenue deltas
 rev_deltas.plot()
 # Mark optimal point
 max_pred = rev_deltas.idxmax()
 plt.scatter(max_pred, rev_deltas.loc[max_pred], s=100, c='red')
 # Reference line for break-even
 plt.hlines(0, 0, 1)
 plt.show()
 print(f'Maximum benefit at cutoff {max_pred} with revenue delta of ${rev_deltas.loc[max_pred]:,.2f}')
plot_tradeoff(rev_deltas)

KeyError: 'predict_prob'